In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score

In [ ]:
# 그리드서치 할 데이터 불러올 것

datasets = [ 
    "데이터이름" , "데이터이름"
]

# 결과 저장 리스트
final_results = []

y=pd.read_csv('scaled_firm_new.csv')[['ret','year']]


for dataset_name in datasets:
    print(f"Processing: {dataset_name}")

    # X 데이터 로드
    X_data = globals()[dataset_name]  # 변수명으로 데이터 로드
    X_data = X_data[(X_data['year'] >= 1997) & (X_data['year'] <= 2016)].drop(columns=['year', 'month'])

    # Y 데이터 (ret와 year
    y_data = y.copy()  # ret와 year가 포함된 데이터프레임
    y_data =y[(y['year'] >= 1997) & (y['year'] <= 2016)]['ret']
    

    # Validation 결과 저장 리스트
    results = []

    # 하이퍼파라미터 그리드 정의
    param_grid = {
        'iterations' : [350, 400, 450],
        'learning_rate': [0.01, 0.03, 0.05],
        'max_depth': [3, 5, 7]
    }

    # TimeSeriesSplit 정의
    tscv = TimeSeriesSplit(n_splits=5)
    model = CatBoostRegressor(verbose=0, random_state = 4)

    # Adjusted R-squared 계산 함수
    def adjusted_r2_score(y_true, y_pred, n, p):
        r2 = r2_score(y_true, y_pred)
        return 1 - (1 - r2) * (n - 1) / (n - p - 1)


    # Custom Scorer 생성 함수
    def adj_r2(estimator, X, y_true):
        n, p = X.shape
        pred = estimator.predict(X)
        return 1 - ((1 - r2_score(y_true, pred)) * (n - 1))/(n-p-1)

    # Scorer를 make_scorer로 생성
    scorer = make_scorer(adj_r2, greater_is_better=True)

    # GridSearchCV 실행
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=tscv,              # TimeSeriesSplit 사용
        scoring = {
           "adj R2": adj_r2
        },
        refit="adj R2",
        verbose=0
    )

    grid_search.fit(X_data, y_data)

    # 최적의 하이퍼파라미터 출력
    print("Best Parameters:", grid_search.best_params_)
    print("Best Score:", grid_search.best_score_)


In [ ]:
X_data = pd.read_csv('C:/Users/yujin/Desktop/계특 플젝 2ver/병합/Firm_AE/AE30F_PCA50E.csv')
X_test = X_data[X_data['year'] == 2017].drop(columns=['year', 'month'])
X_train = X_data[(X_data['year'] >= 1997) & (X_data['year'] <= 2016)].drop(columns=['year', 'month'])

y_data = pd.read_csv('firm_500_new.csv')
y_test =y_data[y_data['year'] == 2017]['ret']
y_train =y_data[(y_data['year'] >= 1997) & (y_data['year'] <= 2016)]['ret']

model = CatBoostRegressor(
    iterations=250,
    learning_rate=0.1,
    max_depth=6,
    verbose=0
)

# Adjusted R-squared 계산 함수
def adjusted_r2_score(y_true, y_pred, n, p):
    r2 = r2_score(y_true, y_pred)
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)


# Custom Scorer 생성 함수
def adj_r2(estimator, X, y_true):
    n, p = X.shape
    pred = estimator.predict(X)
    return 1 - ((1 - r2_score(y_true, pred)) * (n - 1))/(n-p-1)

# Scorer를 make_scorer로 생성
scorer = make_scorer(adj_r2, greater_is_better=True)

model.fit(X_train, y_train)


y_pred = model.predict(X_test)

 
r2 = adjusted_r2_score(y_test, y_pred)
print(r2)